# Problema I

Para esta actividad, vamos a comenzar definiendo los objetos a separar en clústers. Como puede observarse, tanto los datos como los *centroides* se definen como arrays ``numpy`` en 2-D. Este array se puede entender como una *lista de listas* donde cada *sublista* contiene las dos dimensiones de cada punto. 

In [1]:
import numpy as np

centroids = np.array([[2, 10], [8, 4], [5, 8]])
data_points = np.array([[2, 10], [2, 5], [8, 4], 
                        [5, 8], [7, 5], [6, 4], 
                        [1, 2], [4, 9]])

Una vez definidos los datos del problema, pasámos a implementar el algoritmo *K-Folds* que los separará en tantos clústers como indiquemos en el parámetro ``k``. A continuación, se muestra el código realizado junto con una breve explicación de cada funcion mediante comentarios.

In [2]:
# Importamos la implementación de la distáncia Euclidea 
# que realicé en la lección 2
from distances import euclidean

# La función get_centroid recibe como argumento un conjunto de puntos, 
# y aplica la media sobre cada dimensión para obtener el centroide. Es decir,
# entendemos el centroide como la media de los puntos contenidos en data_points
def get_centroid(data_points):
    return np.mean(data_points, axis = 0)

# La función recalculate_centroids recalcula los centroides del conjunto de datos
# data_points, en función del clúster a los que pertenezcan. En esencia, toma los
# puntos que forman parte de un mismo clúster, y calcula su centroide.
def recalculate_centroids(k, data_points, classes):
    centroids = np.zeros(shape = (k, 2))
    for i in range(k):
        pos = np.where(classes == i)
        points = data_points[pos, :][0]
        centroids[i, :] = get_centroid(points)
    return centroids

# Esta función implementa el agrupamiento de un conjunto de puntos (data_points),
# en k clústers, a partir de unos centroides iniciales (centroids)
def k_folds(k, data_points, centroids):
    # Definimos una variable booleana que controla la parada del algoritmo
    # Para esta implementación, consideramos que el algoritmo termina cuando
    # en dos iteraciones seguidas, los puntos no varían el clúster al que pertenecen.
    keep = True
    # Creamos una lista de listas, para representar los centroides en cada iteración
    centroid_list = [centroids]
    clusters = np.zeros(shape = (data_points.shape[0]))
    while keep:
        # Calculamos la distáncia de cada punto a cada uno de los k centroides
        # y las almacenamos en un array numpy 2-D.
        distances = np.array([euclidean(point, centroids) for point in data_points])
        # Cada "sublista" del array, contendrá las distancias de cada punto
        # a cada uno de los centroides. Determinamos el clúster al que pertenece
        # cada punto en función de la mínima distáncia de entre todas las calculadas
        new_clusters = np.argmin(distances, axis = 1)
        # Si los clústers seleccionados no han variado respecto a la iteración
        # anterior, ponemos keep a False para terminar de iterar.
        if np.array_equal(clusters, new_clusters):
            keep = False
        else:
        # En caso contrario, recalculamos los centroides y pasamos a la siguiente iteración
            clusters = new_clusters
            centroids = recalculate_centroids(k, data_points, clusters)
            centroid_list.append(centroids)
    # Devolvemos los clusters de cada punto así como los k centroides en cada iteración
    return new_clusters, np.array(centroid_list)

Una vez visto el funcionamiento del algoritmo, pasamos a llamarlo con los datos definidos.

In [3]:
import pandas as pd

clusters, centroid_list = k_folds(3, data_points, centroids)
print(f'Clústers formados ->')
pd.DataFrame({"Clúster":(clusters+1)}, 
             index = ["A" + str(i) for i in range(1, data_points.shape[0]+1)])

Clústers formados ->


,Clúster
A1,1
A2,3
A3,2
A4,1
A5,2
A6,2
A7,3
A8,1


In [4]:
for centroid, i  in zip(centroid_list, range(centroid_list.shape[0])):
    print(f'Centroides en la iteración {i} ->\n {centroid}\n')

Centroides en la iteración 0 ->
 [[ 2. 10.]
 [ 8.  4.]
 [ 5.  8.]]

Centroides en la iteración 1 ->
 [[ 2.         10.        ]
 [ 7.          4.33333333]
 [ 3.          6.        ]]

Centroides en la iteración 2 ->
 [[3.         9.5       ]
 [7.         4.33333333]
 [2.66666667 5.        ]]

Centroides en la iteración 3 ->
 [[3.66666667 9.        ]
 [7.         4.33333333]
 [1.5        3.5       ]]



Mostramos a continuación la implementación de la función ``SSE`` que, como su nombre indica, nos permite calcular el valor de la métrica SSE de los resultados obtenidos.

In [5]:
def SSE(data_points, centroids, clusters):
    aux_arr = np.array([centroids[i] for i in clusters])
    diff = data_points - aux_arr
    return np.sum(np.square(diff))

print(f'SSE = {SSE(data_points, centroid_list[-1, :], clusters)}')

SSE = 14.333333333333332


Pasamos a calcular los *K-Folds* para el conjunto de datos y centroides inciales, del problema visto en la lección 5.

In [6]:
centroids = np.array([[2, 10], [5, 8], [1, 2]])
data_points = np.array([[2, 10], [2, 5], [8, 4], 
                        [5, 8], [7, 5], [6, 4], 
                        [1, 2], [4, 9]])

clusters, centroid_list = k_folds(3, data_points, centroids)
print(f'Clústers formados ->')
pd.DataFrame({"Clúster":(clusters+1)}, 
             index = ["A" + str(i) for i in range(1, data_points.shape[0]+1)])

Clústers formados ->


,Clúster
A1,1
A2,3
A3,2
A4,1
A5,2
A6,2
A7,3
A8,1


In [7]:
for centroid, i  in zip(centroid_list, range(centroid_list.shape[0])):
    print(f'Centroides en la iteración {i} ->\n {centroid}\n')

Centroides en la iteración 0 ->
 [[ 2. 10.]
 [ 5.  8.]
 [ 1.  2.]]

Centroides en la iteración 1 ->
 [[ 2.  10. ]
 [ 6.   6. ]
 [ 1.5  3.5]]

Centroides en la iteración 2 ->
 [[3.   9.5 ]
 [6.5  5.25]
 [1.5  3.5 ]]

Centroides en la iteración 3 ->
 [[3.66666667 9.        ]
 [7.         4.33333333]
 [1.5        3.5       ]]



In [8]:
print(f'SSE = {SSE(data_points, centroid_list[-1, :], clusters)}')

SSE = 14.333333333333332


Como puede observarse de los resultados obtenidos, vemos como a pesar de usar el mismo conjunto de puntos, se obtienen resultados diferentes. Esto nos permite ver como, efectivamente, la elección de los centroides iniciales resulta clave en los resultados que arroja el problema y, en ciertas circustancias, esto puede resultar contraproducente.  

# Problema II

Antes de comenzar con el agrupamiento jerárquico que propone este ejercicio, vamos a escribir dos funciones que nos van a permitir medir distáncias entre clusters. En concreto, implementaremos el enlace simple (*simple linkage*) y el enlace completo (*complete linkage*).

In [9]:
def simple_linkage(c1, c2, distance):
    distances = np.array([[distance[x1, x2] for x1 in c1] for x2 in c2])
    return np.min(distances)

def complete_linkage(c1, c2, distance):
    distances = np.array([[distance[x1, x2] for x1 in c1] for x2 in c2])
    return np.max(distances)

Una vez implementadas las funciones de distancia, pasamos a implementar el algoritmo que partirá de una matriz de distáncias entre puntos.

In [10]:
def hierarchical_clustering(matrix, simple = True):
    # Implementamos un clustering jerárquico y aglomerativo, es decir,
    # partiremos de un clúster por cada patrón y, en cada paso, fusionaremos los
    # clústers más cercanos. El parámetro booleano simple nos permite controlar
    # si se usará simple_linkage o complete_linkage como función de distancia.
    if simple:
        link_func = simple_linkage
    else:
        link_func = complete_linkage
    # Comenzamos generando un clúster por punto
    clusters = [[i] for i in range(matrix.shape[0])]
    # Generamos una lista de clusters que nos permite controlar la evolución
    # de los mismos durante las iteraciones
    clusters_list = [clusters.copy()]
    # Creamos una copia de la matriz que modificaremos en cada iteración
    aux_mat = matrix.copy()
    while len(clusters) > 1:
        # Calculamos la distancia entre todos los clusters
        dist_clusters = np.array([[link_func(c1, c2, matrix) for c2 in clusters] for c1 in clusters], dtype = float)
        # Reemplazamos los valores 0 (correspondientes a la distáncia de un clúster consigo mismo)
        # por infinito (np.inf)
        np.fill_diagonal(dist_clusters, np.inf)
        # Obtenemos las coordenadas del mínimo (es decir, fila y columna en la matriz)
        min_coord_x, min_coord_y = np.unravel_index(dist_clusters.argmin(), dist_clusters.shape)
        actual_x = set(clusters[min_coord_x])
        actual_y = set(clusters[min_coord_y])
        # Obtenemos de nuevo el listado de clusters
        sub_list_x = [x for x in clusters if actual_x <= set(x)][0]
        sub_list_y = [y for y in clusters if actual_y <= set(y)][0]
        clusters.remove(sub_list_x)
        if sub_list_y in clusters: 
            clusters.remove(sub_list_y)
        clusters.append(list(set(sub_list_x + sub_list_y)))
        clusters_list.append(clusters.copy())
    return clusters_list

In [11]:
matrix = np.array([[0, 1, 2, 9, 10], [1, 0, 3, 7, 5], [2, 3, 0, 4, 6], 
                   [9, 7, 4, 0, 8], [10, 5, 6, 8, 0]])
#matrix = np.array([[0, 1, 4, 5], [1, 0, 2, 6], [4, 2, 0, 3], [5, 6, 3, 0]])
matrix

array([[ 0,  1,  2,  9, 10],
       [ 1,  0,  3,  7,  5],
       [ 2,  3,  0,  4,  6],
       [ 9,  7,  4,  0,  8],
       [10,  5,  6,  8,  0]])

In [12]:
cluster_list_1 = hierarchical_clustering(matrix, simple = True)
aux_map = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E'}
cluster_list_letter = [[[aux_map.get(x) for x in clust] 
                        for clust in clust_iter] 
                       for clust_iter in cluster_list_1]
print('Clusters generados con enlace Simple')
for cluster, i in zip(cluster_list_letter, range(len(cluster_list_letter))):
    print(f'Iteración {i} -> {cluster}')

Clusters generados con enlace Simple
Iteración 0 -> [['A'], ['B'], ['C'], ['D'], ['E']]
Iteración 1 -> [['C'], ['D'], ['E'], ['A', 'B']]
Iteración 2 -> [['D'], ['E'], ['A', 'B', 'C']]
Iteración 3 -> [['E'], ['A', 'B', 'C', 'D']]
Iteración 4 -> [['A', 'B', 'C', 'D', 'E']]


In [13]:
cluster_list_1 = hierarchical_clustering(matrix, simple = False)
cluster_list_letter = [[[aux_map.get(x) for x in clust] 
                        for clust in clust_iter] 
                       for clust_iter in cluster_list_1]
print('Clusters generados con enlace completo\n')
for cluster, i in zip(cluster_list_letter, range(len(cluster_list_letter))):
    print(f'Iteración {i} -> {cluster}')

Clusters generados con enlace completo

Iteración 0 -> [['A'], ['B'], ['C'], ['D'], ['E']]
Iteración 1 -> [['C'], ['D'], ['E'], ['A', 'B']]
Iteración 2 -> [['D'], ['E'], ['A', 'B', 'C']]
Iteración 3 -> [['A', 'B', 'C'], ['D', 'E']]
Iteración 4 -> [['A', 'B', 'C', 'D', 'E']]


Vemos que, el hecho de utilizar enlazado simple o completo, nos genera una discrepancia en la tercera iteración. Este discrepancia se traduce en un dendograma diferente en función del método que utilicemos.

# Problema III

Para este problema, se nos solicitaba partir de los datos del problema I (usaremos la misma definición que se expuso para ese mismo problema) y aplicar el algoritmo DBSCAN con los siguientes parámetros:

- $M = 3$ y $\epsilon = \sqrt{2}$
- $M = 3$ y $\epsilon = \sqrt{10}$

Mostramos a continuación la implementación del algoritmo *DBSCAN* con una serie de comentarios explicativos para su mejor comprensión.

In [14]:
# La función get_clusters_dbscan recibe como argumentos, 
# el conjunto de datos (data_points), el número mínimo de puntos M y
# el radio de cada cluster (eps)
def get_clusters_dbscan(data_points, M, eps):
    # Inicializamos una lísta vacía que contendrá todos los clústers
    # que resulten del proceso
    clusters = []
    # Comenzamos iterando sobre cada punto del conjunto de datos
    for point in data_points:
        # Para cada punto, calculamos la distáncia euclídea con el resto.
        distances = euclidean(point, data_points)
        # Nos quedamos con aquellos puntos qu estén más cercanos que eps
        close = np.where(distances <= eps)[0]
        # Si el número de puntos obtenidos, es mayor o igual al número mínimo M
        if len(close) >= M:
            # Examinamos si el "clúster" definido por el punto actual (point)
            # contiene puntos ya incluidos en alguno de los clústers obtenidos 
            # en iteraciones anteriores. En caso afirmativo, "fusionamos" ambos 
            # conjuntos en un único clúster. Hacemos uso de sets para evitar
            # la repetición de elementos. En caso negativo, el cluster se incluye en la
            # lista "clusters"
            aux_set = set(close)
            added = False
            for i in range(len(clusters)):
                if aux_set & clusters[i] != set():
                    clusters[i] = clusters[i] | aux_set
                    added = True
                    break
            if not added:
                clusters.append(aux_set)
            print(clusters)
    # Devolvemos la lista de clústers
    return clusters

Una vez vista la implementación del algoritmo, pasamos a ejecutarlo con los datos correspondientes.

In [15]:
M1, M2 = 2, 2
eps1, eps2 = np.sqrt(2), np.sqrt(10)
clusters_1 = get_clusters_dbscan(data_points, M1, eps1)
clusters_2 = get_clusters_dbscan(data_points, M2, eps2)

[{2, 4}]
[{2, 4}, {3, 7}]
[{2, 4, 5}, {3, 7}]
[{2, 4, 5}, {3, 7}]
[{2, 4, 5}, {3, 7}]
[{0, 7}]
[{0, 7}, {1, 6}]
[{0, 7}, {1, 6}, {2, 4, 5}]
[{0, 3, 7}, {1, 6}, {2, 4, 5}]
[{0, 3, 7}, {1, 6}, {2, 4, 5}]
[{0, 3, 7}, {1, 6}, {2, 4, 5}]
[{0, 3, 7}, {1, 6}, {2, 4, 5}]
[{0, 3, 7}, {1, 6}, {2, 4, 5}]


Para una correcta interpretación de los resultados, debemos tener en cuenta que, si un punto no se ha situado en ningún clúster, aparecerá con el valor reservado ``NaN`` en el *DataFrame* que mostaremos a continuación.

In [16]:
clusters_aux = np.full(shape = (data_points.shape[0], ), fill_value = np.nan)
for cluster, i in zip(clusters_1, range(len(clusters_1))):
    clusters_aux[list(cluster)] = i

print(f'Resultados con M = {M1} y epsilon = {eps1}')
pd.DataFrame({"Clúster":(clusters_aux+1)}, 
             index = ["A" + str(i) for i in range(1, data_points.shape[0]+1)])

Resultados con M = 2 y epsilon = 1.4142135623730951


,Clúster
A1,NaN
A2,NaN
A3,1.0
A4,2.0
A5,1.0
A6,1.0
A7,NaN
A8,2.0


In [17]:
clusters_aux = np.full(shape = (data_points.shape[0], ), fill_value = np.nan)
for cluster, i in zip(clusters_2, range(len(clusters_2))):
    clusters_aux[list(cluster)] = i

print(f'Resultados con M = {M2} y epsilon = {eps2}')
pd.DataFrame({"Clúster":(clusters_aux+1)}, 
             index = ["A" + str(i) for i in range(1, data_points.shape[0]+1)])

Resultados con M = 2 y epsilon = 3.1622776601683795


,Clúster
A1,1.0
A2,2.0
A3,3.0
A4,1.0
A5,3.0
A6,3.0
A7,2.0
A8,1.0
